In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#loading data
train = pd.read_csv('astro_train.csv')
test = pd.read_csv('astro_test.csv')
train.head()

,id,dered_i,dered_z,dered_u,dered_g,dered_r,err_i,err_z,err_u,err_g,...,skyVersion,run,rerun,camCol,field,obj,photoz,#ra,dec,class
0,0,15.1459,14.9723,18.6394,16.4080,15.4896,0.00195,0.00406,0.02796,0.00279,...,1,745,40,2,518,5,2.64988,236.22077,-0.58795,0
1,1,17.3579,17.3498,18.5273,17.6527,17.4296,0.00757,0.02498,0.02484,0.00536,...,1,745,40,2,518,8,2.63438,236.22135,-0.46320,0
2,2,20.6154,19.7785,25.4019,23.4824,22.5504,0.10071,0.16768,0.75381,0.46380,...,1,745,40,2,518,9,1.65738,236.22055,-0.46434,0
3,3,15.0188,14.9927,16.7971,15.5712,15.1534,0.00183,0.00411,0.00820,0.00195,...,1,745,40,2,518,12,2.28431,236.22224,-0.61553,0
4,4,20.3008,21.6454,23.7314,21.7139,20.9195,0.21194,1.86698,2.73388,0.27544,...,1,745,40,2,518,13,1.34673,236.22307,-0.61710,1


In [3]:
#preprocessing dataset
y_train= train['class']
train = train.drop('class', axis= 1)

dataset = pd.concat([train, test])
dataset = dataset.drop(['id','skyVersion', 'camCol', 'run', 'rerun'], axis= 1)

dataset["err_g_cbrt"] = np.cbrt(dataset["err_g"])
dataset["err_i_cbrt"] = np.cbrt(dataset["err_i"])
dataset["err_r_cbrt"] = np.cbrt(dataset["err_r"])
dataset["err_u_cbrt"] = np.cbrt(dataset["err_u"])
dataset["err_z_cbrt"] = np.cbrt(dataset["err_z"])


dataset = dataset.drop(['err_g', 'err_i', 'err_r', 'err_u','err_z','#ra', 'field', 'obj', 'dec'], axis= 1)
ss = dataset.iloc[:, :].values

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
transformed_dataset = sc.fit_transform(ss)

train_prepared = transformed_dataset[:45000, :]
test_prepared = transformed_dataset[45000:, :]

In [4]:
#over-sampling using SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_res, y_res = smote.fit_sample(train_prepared, y_train)

Using TensorFlow backend.


In [5]:
#creating model using random forest classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(bootstrap=False, n_estimators = 600,criterion = 'entropy', random_state = 0)
classifier.fit(x_res, y_res)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [6]:
#spliting data into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_res, y_res, test_size=0.2)

#checking accuracies using cross val score
from sklearn.model_selection import cross_val_score
accurarices = cross_val_score(classifier, X_train, Y_train, cv= 3, n_jobs= -1)
accurarices

array([0.73809829, 0.72685274, 0.73192172])

In [7]:
#predicting output
y_pre = classifier.predict(test_prepared)

In [8]:
#saving output in a csv file
y = pd.DataFrame(y_pre)
y.to_csv('/content/drive/My Drive/filename.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/filename.csv'